In [42]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import math
import torch
import torch.nn as nn
import torch.nn.functional as F
import random
import torch.optim as optim
from utils import *

Defaulting to user installation because normal site-packages is not writeable
  Using cached pyod-0.3.4.tar.gz (194 kB)
  Preparing metadata (setup.py) ... error
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> [12 lines of output]
      Traceback (most recent call last):
        File "<string>", line 2, in <module>
        File "<pip-setuptools-caller>", line 34, in <module>
        File "/private/var/folders/jt/l64gyc6x6d591spvk3ctx7vm0000gn/T/pip-install-dm7mxakw/pyod_b3b960d5462a4430ab5d560304e48b08/setup.py", line 2, in <module>
          from pyod import __version__
        File "/private/var/folders/jt/l64gyc6x6d591spvk3ctx7vm0000gn/T/pip-install-dm7mxakw/pyod_b3b960d5462a4430ab5d560304e48b08/pyod/__init__.py", line 3, in <module>
          from . import models
        File "/private/var/folders/jt/l64gyc6x6d591spvk3ctx7vm0000gn/T/pip-install-dm7mxakw/pyod_b3b960d5462a4430ab5d560304e48b08/pyod/models/__init__.

In [2]:
X, Y, S = read_h5py('SoliData.zip', reshaped=True)

In [20]:
X[0].shape

(68, 32, 32)

In [34]:
def aply_cov(X, method="easy_sum"):
    X_cov = []
    if (method=="join_sum"):
        '''
            on effectue la covariance sur la jointure des matrice de chaque x_i 
            on considere que chaque x_i contient une matrice
            ex : x_i = [[1,2,3], [3, 4, 5]]
            alors on considere que x_i = [1, 2, 3, 4, 4] et on calcul la cov sur cela
        '''
        for x in X:
            #merging...
            x_merged = x[0].T
            for x_ in x[1:]:
                x_merged = np.concatenate((x_merged, x_.T), axis=1)
            #calculate the cov
            X_cov.append(np.cov(x_merged))
    elif (method=="easy_sum"):
        for x in X:
            cov = np.zeros(shape=(32, 32))
            for x_ in x:
                cov = cov + np.cov(x_.T)
            X_cov.append(cov*(1/x.shape[0]))
    elif (method=="temp_sum"):
        '''
            par matrice temporelle
            C_t = somme(cov(x_Td))
        '''
        for x in X:
            cov = np.zeros(shape=x.shape[0])
            for x_ in x:
                c_t = 0
                for x_i in x_:
                    c_t = c_t + np.cov(x_i)
                cov = np.append(cov, c_t*(1/x.shape[0]))
            X_cov.append(cov)
            
    return np.array(X_cov, dtype="object")

In [40]:
data_augment  = data_augment(X, Y, S, pourcentage=1)
X_augmented = np.concatenate((X, data_augment[0]), axis=0) 
Y_augmented = np.concatenate((Y, data_augment[1]), axis=0)
S_augmented = np.concatenate((S, data_augment[2]), axis=0)

In [36]:
x = aply_cov(X, method="temp_sum")

KeyboardInterrupt: 